In [19]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import joblib

In [25]:
dataset_url = 'https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv'
df = pd.read_csv(dataset_url).drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [26]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise




In [27]:
df = normalize_text(df)
df.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [28]:
df['sentiment'].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [29]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]


In [30]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
df.head()


C:\Users\satya\AppData\Local\Temp\ipykernel_33872\3057201912.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im not thinking old friend want married ...
8,0,charviray charlene love miss
9,0,kelcouch sorry least friday


In [31]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
import dagshub


mlflow.set_tracking_uri('https://dagshub.com/satyajeetrai007/Mlops-Project1.mlflow')
dagshub.init(repo_owner='satyajeetrai007', repo_name='Mlops-Project1', mlflow=True)


mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "satyajeetrai007/Mlops-Project1"

Repository satyajeetrai007/Mlops-Project1 initialized!

<Experiment: artifact_location='mlflow-artifacts:/dda290146cdb41e79ac32d3ab71bf94d', creation_time=1753292318622, experiment_id='0', last_update_time=1753292318622, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [45]:
import mlflow.data.pandas_dataset

with mlflow.start_run():
    
    # Log preprocessing parameters
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param("num_features", 1000)
    mlflow.log_param("test_size", 0.2)
    
    # Model building and training
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # Log model parameters
    mlflow.log_param("model", "Logistic Regression")
    
    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Log evaluation metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    # Log model
    print(f"mlflow version is {mlflow.__version__}")
    mlflow.sklearn.log_model(model,"model")# this is not working right now!
    
    model_path = r"lr_bow_model.pkl"
    joblib.dump(model, model_path)
    mlflow.log_artifact(model_path,"lr_bow_model_artifact")



    # Log train and test data (data versioning)
    df_data = mlflow.data.from_pandas(df,source =dataset_url)

   
   

    mlflow.log_input(df_data,"X_train")
  

    # Save and log the notebook
    import os
    notebook_path = "exp1_baseline_model.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)
    
    # Print the results for verification
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

2025/07/24 19:38:56 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
2025/07/24 19:38:57 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during sklearn autologging: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
2025/07/24 19:39:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


mlflow version is 3.1.4
🏃 View run resilient-ape-729 at: https://dagshub.com/satyajeetrai007/Mlops-Project1.mlflow/#/experiments/0/runs/2947f0463f4444be954843bc2e2836ee
🧪 View experiment at: https://dagshub.com/satyajeetrai007/Mlops-Project1.mlflow/#/experiments/0


RestException: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}